In [1]:
# data frames/structures
import pandas as pd
import numpy as np

# pytorch - neural network library
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# custom class used to define the LossTriangleClassifier network
from model.LossTriangleClassifier import LossTriangleClassifier

In [3]:
# Triangle class imported from (future) rocky package
from triangle import Triangle

In [4]:
# methods for working with excel files
import openpyxl
from openpyxl.utils import range_to_tuple

In [5]:
# pre-fit/saved triangle model
model_file = r"C:\Users\AndyW\OneDrive\Documents\inc_cum_tri.torch"

In [6]:
# load paid & reported loss triangles, cumulative and incurred, from csu link ratio file
# using Triangle.from_excel() method

filename = r"C:\Users\AndyW\OneDrive\work\2022Q4 - OL Occ.xlsx"
rpt_sheet = "Reported Loss Development"
paid_sheet = "Paid Loss Development"

cum_rng = "B5:CD25"
inc_rng = "B83:CD103"

In [7]:
r = f"'{rpt_sheet}'!{cum_rng}"

print(f"range:\n{r}\n")
print(f"Running through `range_to_tuple` method yields:\n{range_to_tuple(r)}")

range:
'Reported Loss Development'!B5:CD25

Running through `range_to_tuple` method yields:
('Reported Loss Development', (2, 5, 82, 25))


In [8]:
rpt_cum = Triangle.from_clipboard(id='rpt_loss', origin_columns=1)
rpt_cum

0            3         6         9        12          15          18  \
AY                                                                     
2003                                                                   
2004                                                                   
2005                                                                   
2006                                                                   
2007                                                                   
2008      -      224.22     97.59    521.49    1,001.83    1,004.08    
2009   307.97    254.44    347.82    368.09      449.19      460.47    
2010   211.50    267.20    465.57    407.76      451.49      484.79    
2011   287.87    282.87    315.02    425.71      438.80      465.27    
2012   159.26    216.23    303.30    313.43      341.46      376.77    
2013   156.30    232.44    275.16    275.56      297.84      318.48    
2014   276.27    284.44    281.10    281.06      315.44      325

### preprocess data:
1. Remove AY < 2008 that are all blank
2. convert to 10x10 triangle
3. reshape as if it were a standard yearly triangle (just remove the extra columns so that it is a 10x10 triangle)

In [19]:
# 1. remove AY<2008
df = (rpt_cum.triangle
      
      .assign(AY=lambda x: x.index.astype(int)) # convert index to integer (right now it is likely a string)
      .query('AY >= 2008'))                     # only take AY >= 2008
df.iloc[:, :15]

,3,6,9,12,15,18,21,24,27,30,33,36,39,42,45
AY,,,,,,,,,,,,,,,
2008,-,224.22,97.59,521.49,"1,001.83","1,004.08","1,065.65",973.26,"1,066.52","1,100.35","1,043.07",982.84,971.15,"1,009.13","1,055.30"
2009,307.97,254.44,347.82,368.09,449.19,460.47,472.95,424.28,424.13,423.60,416.49,363.98,361.43,344.64,368.21
2010,211.50,267.20,465.57,407.76,451.49,484.79,491.17,481.22,488.62,478.89,478.94,459.43,437.42,488.64,423.83
2011,287.87,282.87,315.02,425.71,438.80,465.27,469.73,461.11,464.61,457.61,482.69,436.01,417.36,399.74,378.47
2012,159.26,216.23,303.30,313.43,341.46,376.77,360.86,354.13,346.83,334.59,345.85,331.67,321.54,322.03,302.26
2013,156.30,232.44,275.16,275.56,297.84,318.48,316.93,346.47,363.89,369.74,370.22,359.37,357.76,366.82,363.16
2014,276.27,284.44,281.10,281.06,315.44,325.81,363.42,350.31,378.70,383.01,394.45,380.79,374.83,387.04,367.66
2015,288.18,219.54,291.58,274.94,300.05,296.77,281.77,289.59,318.02,332.81,339.17,349.00,347.77,355.48,352.33
2016,182.46,193.84,261.68,239.19,267.99,327.83,326.00,340.67,337.24,354.05,367.28,373.03,373.19,368.94,378.35


In [20]:
# 2. convert to 10x10 -- should be first 10 columns, last 10 rows
df = df.iloc[-10:, :10]
df

,3,6,9,12,15,18,21,24,27,30
AY,,,,,,,,,,
2013,156.30,232.44,275.16,275.56,297.84,318.48,316.93,346.47,363.89,369.74
2014,276.27,284.44,281.10,281.06,315.44,325.81,363.42,350.31,378.70,383.01
2015,288.18,219.54,291.58,274.94,300.05,296.77,281.77,289.59,318.02,332.81
2016,182.46,193.84,261.68,239.19,267.99,327.83,326.00,340.67,337.24,354.05
2017,159.34,188.92,223.63,234.24,279.71,309.97,324.84,339.03,368.96,402.19
2018,184.90,239.71,244.78,238.83,283.72,317.82,347.43,359.25,389.21,399.33
2019,176.67,223.21,272.60,258.66,317.22,348.26,377.44,415.67,417.33,462.90
2020,157.68,204.30,306.83,294.99,322.28,372.77,387.66,422.06,446.16,456.83
2021,222.35,354.80,341.57,360.20,427.79,499.01,541.86,574.79,,
